In [ ]:
# ============================================================================
# Uni2TS Large Portfolio Backtesting - Google Colab (A100 bf16 auto)
# ============================================================================

from google.colab import drive
drive.mount('/content/drive')

!mkdir -p "/content/drive/MyDrive/uni2ts_large_portfolio(FineTuning)"

%cd "/content/drive/MyDrive/uni2ts_large_portfolio(FineTuning)"

!git clone https://github.com/SalesforceAIResearch/uni2ts
%cd uni2ts

%pip install torch transformers scikit-learn tqdm joblib gluonts lightning jaxtyping hydra-core peft statsmodels scipy yfinance einops huggingface_hub optuna

import torch
print("CUDA is available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("Current CUDA device:", torch.cuda.current_device())
    print("Device name:", torch.cuda.get_device_name(0))
import sys
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
import time
import inspect
import os
import joblib
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from gluonts.dataset.common import ListDataset
sys.path.append("src")
from uni2ts.model.moirai import MoiraiForecast, MoiraiModule
import random
import yfinance as yf
import statsmodels.api as sm
from scipy.stats import f as f_dist
import matplotlib.dates as mdates
from datetime import datetime
from typing import Dict, Any, Optional, List
from torch import nn
from torch.utils.data import Dataset, DataLoader
from peft import LoraConfig, get_peft_model, PeftModel, TaskType
from uni2ts.model.moirai.finetune import MoiraiFinetune
torch.manual_seed(42)
np.random.seed(42)
random.seed(42)

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

USE_BF16 = False
GPU_NAME = ""
if torch.cuda.is_available():
    try:
        GPU_NAME = torch.cuda.get_device_name(0)
        if ("A100" in GPU_NAME) or ("H100" in GPU_NAME):
            USE_BF16 = True
    except Exception:
        pass
print(f"GPU: {GPU_NAME}")
print(f"Enable bf16: {USE_BF16}")

MODEL_NAME = "Salesforce/moirai-1.1-R-large"
PRED_LEN = 1
PATCH_SIZE = "auto"
WINDOW_SIZES = [5, 21, 252, 512]
START_YEAR = 2016
NPZ_PATH = "/content/drive/MyDrive/ERP Data/all_window_datasets_unscaled.npz"

LAST_STEP_ONLY = True  # If True: only calculate loss at the last PRED_LEN step; False: keep current masked reconstruction

BASE_MODEL_ID = MODEL_NAME
WINDOWS = WINDOW_SIZES

print(f"Model: {MODEL_NAME}")
print(f"Prediction length: {PRED_LEN}")

data_path = NPZ_PATH
if os.path.exists(data_path):
    data = np.load(data_path, allow_pickle=True)
    print("Data loaded successfully!")
else:
    print(f"Data file not found: {data_path}")
    print("Please ensure all_window_datasets.npz is uploaded to Google Drive 'ERP Data' folder")

window_sizes = [5, 21, 252, 512]
results = {}

def load_npz_dataset(npz_path: str) -> Dict[str, Any]:
    d = np.load(npz_path, allow_pickle=True)
    return {k: d[k] for k in d.files}

def extract_split(data: Dict[str, Any], window: int, split: str):
    X = data[f"X_{split}_{window}"]
    y = data[f"y_{split}_{window}"]
    meta_raw = data.get(f"meta_{split}_{window}")
    if meta_raw is None:
        meta = pd.DataFrame({"PERMNO": np.arange(len(X))})
    else:
        meta = pd.DataFrame(meta_raw.item()) if hasattr(meta_raw, "item") else pd.DataFrame(meta_raw)
    return X, y, meta

def time_based_val_split(X: np.ndarray, y: np.ndarray, meta: pd.DataFrame, val_ratio: float = 0.2):
    n = len(X)
    val_start = int(np.floor(n * (1.0 - val_ratio)))
    X_tr, y_tr = X[:val_start], y[:val_start]
    X_va, y_va = X[val_start:], y[val_start:]
    meta_tr = meta.iloc[:val_start].reset_index(drop=True)
    meta_va = meta.iloc[val_start:].reset_index(drop=True)
    return (X_tr, y_tr), (X_va, y_va), (meta_tr, meta_va)

data_all = load_npz_dataset(NPZ_PATH) if os.path.exists(NPZ_PATH) else {}

def get_batch_size(window_size):
    """
    Batch size configuration for T4 GPU (16GB).
    Uni2TS Large model requires smaller batch size.
    """
    if window_size <= 5:
        return 1024
    elif window_size <= 21:
        return 512
    elif window_size <= 252:
        return 128
    elif window_size <= 512:
        return 64
    else:
        return 4

print("T4 GPU optimized batch size configuration for Uni2TS Large:")
for ws in WINDOW_SIZES:
    batch_size = get_batch_size(ws)
    print(f"  Window {ws}: batch size = {batch_size}")

Mounted at /content/drive
/content/drive/MyDrive/uni2ts_large_portfolio(FineTuning)
Cloning into 'uni2ts'...
remote: Enumerating objects: 1114, done.
remote: Counting objects: 100% (565/565), done.
remote: Compressing objects: 100% (260/260), done.
remote: Total 1114 (delta 403), reused 305 (delta 305), pack-reused 549 (from 2)
Receiving objects: 100% (1114/1114), 9.47 MiB | 16.61 MiB/s, done.
Resolving deltas: 100% (550/550), done.
Updating files: 100% (268/268), done.
/content/drive/MyDrive/uni2ts_large_portfolio(FineTuning)/uni2ts
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 125.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 88.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 60.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 1

In [ ]:

def annual_sharpe(rets, freq=252):
    mu = float(np.mean(rets)) * freq
    sd = float(np.std(rets, ddof=1)) * np.sqrt(freq)
    return mu / sd if sd > 0 else 0

# === Load risk-free rate & calculate S&P500 Excess Sharpe ===

rf_file = "/content/drive/MyDrive/ERP Data/CRSP_2016_2024_top50_with_exret.csv"
try:
    rf_df = pd.read_csv(rf_file, usecols=["date", "rf"])
    rf_df["date"] = pd.to_datetime(rf_df["date"])
    rf_df = rf_df.drop_duplicates("date").set_index("date").sort_index()
    rf_series = rf_df["rf"].astype(float)

    px = yf.download("^GSPC", start="2016-01-01", end="2024-12-31")["Close"]
    sp_ret = px.pct_change().dropna()
    rf_align = rf_series.reindex(sp_ret.index).fillna(method="ffill")
    sp_excess = sp_ret.values - rf_align.values

    SR_MKT_EX = annual_sharpe(sp_excess)
    print(f"[INFO] S&P500 Excess Sharpe (2016–24) = {SR_MKT_EX:.3f}")
except Exception as e:
    print(f"Warning: Could not load risk-free rate data: {e}")
    SR_MKT_EX = 0.5  # Use default value

# === 2. ΔSharpe calculation function ===
def delta_sharpe(r2_zero: float, sr_base: float):
    """
    If r2_zero <= 0   → ΔSharpe = 0, Sharpe* = sr_base
    If r2_zero >= 1   → ΔSharpe = 0, Sharpe* = sr_base (extreme case fallback)
    Otherwise, use the original formula
    """
    if (r2_zero <= 0) or (r2_zero >= 1):
        return 0.0, sr_base
    sr_star = np.sqrt(sr_base ** 2 + r2_zero) / np.sqrt(1 - r2_zero)
    return sr_star - sr_base, sr_star

# === Zero-based R² ===
def r2_zero(y_true, y_pred):
    """
    Calculate zero-based R² (baseline is 0)
    y_true: actual values array (N,)
    y_pred: predicted values array (N,)
    """
    rss = np.sum((y_true - y_pred)**2)
    tss = np.sum(y_true**2)
    return 1 - rss / tss

def calc_ic_daily(df, method='spearman'):
    """
    Calculate daily cross-sectional RankIC
    df: must contain ['signal_date','y_true','y_pred']
    """
    ics = (df.groupby('signal_date')
             .apply(lambda g: g['y_pred'].corr(g['y_true'], method=method))
             .dropna())
    mean_ic = ics.mean()
    std_ic  = ics.std(ddof=1)
    t_ic    = mean_ic / (std_ic / np.sqrt(len(ics))) if std_ic > 0 else np.nan
    pos_ratio = (ics > 0).mean()
    return mean_ic, t_ic, pos_ratio, ics

def calc_directional_metrics(y_true, y_pred, permnos=None):
    """
    - Sample-level sign prediction
    - If grouped by stock, calculate Overall, Up, Down for each stock and then average
    """
    y_true = np.asarray(y_true)
    y_pred = np.asarray(y_pred)

    if permnos is None:
        s_true = np.sign(y_true)
        s_pred = np.sign(y_pred)
        mask = s_true != 0
        s_true = s_true[mask]
        s_pred = s_pred[mask]

        overall_acc = np.mean(s_true == s_pred)

        up_mask = s_true > 0
        down_mask = s_true < 0
        up_acc = np.mean(s_true[up_mask] == s_pred[up_mask]) if np.any(up_mask) else 0
        down_acc = np.mean(s_true[down_mask] == s_pred[down_mask]) if np.any(down_mask) else 0

    else:
        df = pd.DataFrame({"permno": permnos, "yt": y_true, "yp": y_pred})
        overall_accs = []
        up_accs = []
        down_accs = []

        for _, g in df.groupby("permno"):
            s_true = np.sign(g["yt"].values)
            s_pred = np.sign(g["yp"].values)
            mask = s_true != 0
            s_true = s_true[mask]
            s_pred = s_pred[mask]
            if len(s_true) == 0:
                continue
            overall_accs.append(np.mean(s_true == s_pred))

            up_mask = s_true > 0
            down_mask = s_true < 0
            up_accs.append(np.mean(s_true[up_mask] == s_pred[up_mask]) if np.any(up_mask) else np.nan)
            down_accs.append(np.mean(s_true[down_mask] == s_pred[down_mask]) if np.any(down_mask) else np.nan)

        overall_acc = np.nanmean(overall_accs)
        up_acc = np.nanmean(up_accs)
        down_acc = np.nanmean(down_accs)

    return overall_acc, up_acc, down_acc

# === Combined regression metrics ===
def regression_metrics(y_true, y_pred, k, meta=None, permnos=None):
    """
    Includes:
    - Regression metrics
    - Pointwise directional accuracy
    - Market cap group metrics
    """
    y_true = np.asarray(y_true)
    y_pred = np.asarray(y_pred)
    n = len(y_true)

    r2 = r2_zero(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    mae = mean_absolute_error(y_true, y_pred)
    mse = mean_squared_error(y_true, y_pred)

    dir_acc, up_acc, down_acc = calc_directional_metrics(y_true, y_pred, permnos)

    metrics = {
        "R²_zero": r2,
        "RMSE": rmse,
        "MAE": mae,
        "MSE": mse,
        "Directional Accuracy": dir_acc,
        "Up_Directional_Acc": up_acc,
        "Down_Directional_Acc": down_acc
    }

    if meta is not None and "MKTCAP_PERCENTILE" in meta:
        top_mask = meta["MKTCAP_PERCENTILE"] >= 0.75
        bottom_mask = meta["MKTCAP_PERCENTILE"] <= 0.25

        if np.any(top_mask):
            yt_top = y_true[top_mask]
            yp_top = y_pred[top_mask]
            perm_top = permnos[top_mask] if permnos is not None else None
            r2_top = r2_zero(yt_top, yp_top)
            rmse_top = np.sqrt(mean_squared_error(yt_top, yp_top))
            mae_top = mean_absolute_error(yt_top, yp_top)
            mse_top = mean_squared_error(yt_top, yp_top)
            dir_top, up_top, down_top = calc_directional_metrics(yt_top, yp_top, perm_top)
            metrics.update({
                "Top25_R2_zero": r2_top,
                "Top25_RMSE": rmse_top,
                "Top25_MAE": mae_top,
                "Top25_MSE": mse_top,
                "Top25_Dir_Acc": dir_top,
                "Top25_Up_Acc": up_top,
                "Top25_Down_Acc": down_top
            })

        if np.any(bottom_mask):
            yt_bot = y_true[bottom_mask]
            yp_bot = y_pred[bottom_mask]
            perm_bot = permnos[bottom_mask] if permnos is not None else None
            r2_bot = r2_zero(yt_bot, yp_bot)
            rmse_bot = np.sqrt(mean_squared_error(yt_bot, yp_bot))
            mae_bot = mean_absolute_error(yt_bot, yp_bot)
            mse_bot = mean_squared_error(yt_bot, yp_bot)
            dir_bot, up_bot, down_bot = calc_directional_metrics(yt_bot, yp_bot, perm_bot)
            metrics.update({
                "Bottom25_R2_zero": r2_bot,
                "Bottom25_RMSE": rmse_bot,
                "Bottom25_MAE": mae_bot,
                "Bottom25_MSE": mse_bot,
                "Bottom25_Dir_Acc": dir_bot,
                "Bottom25_Up_Acc": up_bot,
                "Bottom25_Down_Acc": down_bot
            })

    return metrics

def f_statistic(y_true, y_pred, k):
    """Return F statistic and corresponding p-value"""
    n   = len(y_true)
    rss = np.sum((y_true - y_pred) ** 2)
    tss = np.sum(y_true ** 2)
    r2  = 1 - rss / tss
    if (r2 <= 0) or (n <= k):
        return 0.0, 1.0
    F = (r2 / k) / ((1 - r2) / (n - k))
    p = f_dist.sf(F, k, n - k)
    return F, p

def overall_interval_metrics_method1(y_all, yhat_all, k, permnos_all=None, meta_all=None):
    """
    Method 1: Calculate metrics for the entire interval at once (concatenate all samples from 2016-2024)
    Returns: a dict, can be directly used for save_metrics()
    """
    base = regression_metrics(
        y_true=y_all,
        y_pred=yhat_all,
        k=k,
        meta=meta_all,
        permnos=permnos_all
    )
    F, p = f_statistic(y_all, yhat_all, k)
    base["F_stat"]     = F
    base["F_pvalue"]   = p
    base["N_obs"] = len(y_all)

    delta_cash, sr_star_cash = delta_sharpe(base["R²_zero"], sr_base=0)
    base["ΔSharpe_cash"]      = delta_cash
    base["Sharpe*_cash"]      = sr_star_cash

    delta_mkt , sr_star_mkt  = delta_sharpe(base["R²_zero"], sr_base=SR_MKT_EX)
    base["ΔSharpe_mkt"]       = delta_mkt
    base["Sharpe*_mkt"]       = sr_star_mkt

    return base

def sortino_ratio(rets, freq=252):
    """Calculate Sortino Ratio"""
    downside = rets[rets < 0]
    if len(downside) == 0:
        return np.inf
    mu = rets.mean() * freq
    sigma = np.sqrt((downside ** 2).mean()) * np.sqrt(freq)
    return mu / sigma

def cvar(rets, alpha=0.95):
    """Calculate CVaR"""
    q = np.quantile(rets, 1 - alpha)
    return rets[rets <= q].mean()

def save_metrics(metrics_dict, name, window, path="portfolio_metrics.csv"):
    """Save metrics to CSV file"""
    row = {'Model': name, 'Window': window}
    row.update(metrics_dict)

    if os.path.exists(path):
        df = pd.read_csv(path)
        df = pd.concat([df, pd.DataFrame([row])], ignore_index=True)
    else:
        df = pd.DataFrame([row])

    df.to_csv(path, index=False)
    print(f"Metrics saved for {name}_w{window} to {path}")

TC_GRID = [0.0005, 0.001, 0.002, 0.003, 0.004]  # 5, 10, 20, 30, 40 bps
TC_TAG  = {
    0.0005: "tc5",
    0.001:  "tc10",
    0.002:  "tc20",
    0.003:  "tc30",
    0.004:  "tc40"
}

class PortfolioBacktester:
    def __init__(self):
        self.results = {}

    def calc_turnover(self, w_t, r_t, w_tp1):
        """Calculate turnover using the standard formula"""
        if w_t is None:
            return np.sum(np.abs(w_tp1))

        gross_ret = np.sum(w_t * r_t)
        if abs(1 + gross_ret) < 1e-8:
            return np.sum(np.abs(w_tp1))

        passive_weight = w_t * (1 + r_t) / (1 + gross_ret)
        turnover = np.sum(np.abs(w_tp1 - passive_weight))
        return turnover

    def create_portfolios_with_permno_tracking(self, signals, market_caps, permnos, top_pct=0.1, bottom_pct=0.1, weight_scheme="VW"):
        """
        Create portfolio weights based on signals, strictly tracking permno alignment.
        weight_scheme: 'VW' value-weighted, 'EW' equal-weighted
        """
        n_stocks = len(signals)
        top_n    = max(1, int(round(n_stocks * top_pct)))
        bottom_n = max(1, int(round(n_stocks * bottom_pct)))

        sorted_idx = np.argsort(signals)[::-1]

        top_idx = sorted_idx[:top_n]
        bottom_idx = sorted_idx[-bottom_n:]

        portfolio_data = {}

        long_weights = np.zeros(n_stocks)
        if len(top_idx) > 0:
            if weight_scheme == "VW":
                top_market_caps = market_caps[top_idx]
                if np.sum(top_market_caps) > 0:
                    long_weights[top_idx] = top_market_caps / np.sum(top_market_caps)
            else:
                long_weights[top_idx] = 1.0 / len(top_idx)

        portfolio_data['long_only'] = {
            'weights': long_weights,
            'permnos': permnos.copy(),
            'selected_permnos': permnos[top_idx] if len(top_idx) > 0 else np.array([])
        }

        short_weights = np.zeros(n_stocks)
        if len(bottom_idx) > 0:
            if weight_scheme == "VW":
                bottom_market_caps = market_caps[bottom_idx]
                if np.sum(bottom_market_caps) > 0:
                    short_weights[bottom_idx] = -bottom_market_caps / np.sum(bottom_market_caps)
            else:
                short_weights[bottom_idx] = -1.0 / len(bottom_idx)

        portfolio_data['short_only'] = {
            'weights': short_weights,
            'permnos': permnos.copy(),
            'selected_permnos': permnos[bottom_idx] if len(bottom_idx) > 0 else np.array([])
        }

        ls_raw = long_weights + short_weights

        gross_target = 2.0
        current_gross = np.sum(np.abs(long_weights)) + np.sum(np.abs(short_weights))
        scale = gross_target / current_gross if current_gross > 1e-8 else 0.0
        ls_weights = scale * ls_raw

        ls_selected_permnos = np.concatenate([
            permnos[top_idx] if len(top_idx) > 0 else np.array([]),
            permnos[bottom_idx] if len(bottom_idx) > 0 else np.array([])
        ])

        portfolio_data['long_short'] = {
            'weights': ls_weights,
            'permnos': permnos.copy(),
            'selected_permnos': ls_selected_permnos
        }

        return portfolio_data

    def calculate_aligned_portfolio_return(self, portfolio_weights, portfolio_permnos, actual_returns, actual_permnos):
        """Calculate portfolio return strictly aligned by permno"""
        aligned_returns = np.zeros(len(portfolio_permnos))

        return_dict = dict(zip(actual_permnos, actual_returns))

        for i, permno in enumerate(portfolio_permnos):
            if permno in return_dict:
                aligned_returns[i] = return_dict[permno]

        portfolio_return = np.sum(portfolio_weights * aligned_returns)
        return portfolio_return, aligned_returns

    def calculate_metrics(self, returns, turnover_series=None):
        """Calculate portfolio metrics - returns summary metrics only, not full series"""
        returns = np.array(returns)

        annual_return = np.mean(returns) * 252
        annual_vol = np.std(returns, ddof=1) * np.sqrt(252)
        sharpe = annual_return / annual_vol if annual_vol > 0 else 0

        log_cum = np.cumsum(np.log1p(returns))
        peak_log = np.maximum.accumulate(log_cum)
        dd_log = peak_log - log_cum
        max_drawdown = 1 - np.exp(-dd_log.max())
        max_1d_loss = np.min(returns)

        avg_turnover = np.mean(turnover_series) if turnover_series is not None else 0

        sortino = sortino_ratio(returns)
        cvar95  = cvar(returns, alpha=0.95)

        result = {
            'annual_return': annual_return,
            'annual_vol': annual_vol,
            'sharpe': sharpe,
            'max_drawdown': max_drawdown,
            'max_1d_loss': max_1d_loss,
            'avg_turnover': avg_turnover,
            'sortino': sortino,
            'cvar95': cvar95
        }

        return result


/tmp/ipython-input-1442714245.py:16: FutureWarning: YF.download() has changed argument auto_adjust default to True
  px = yf.download("^GSPC", start="2016-01-01", end="2024-12-31")["Close"]
[*********************100%***********************]  1 of 1 completed

[INFO] S&P500 Excess Sharpe (2016–24) = 0.652



/tmp/ipython-input-1442714245.py:18: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  rf_align = rf_series.reindex(sp_ret.index).fillna(method="ffill")


In [ ]:
# ===== q/k/v naming inspection (print Moirai attention module field names) =====
from uni2ts.module.attention import GroupedQueryAttention

# Typical module names for LoRA targets (q/k/v/out/ffn)
LORA_DEFAULT_TARGETS = [
    "q_proj", "k_proj", "v_proj", "out_proj",  # attention projections
    "fc1", "fc2"  # FFN layers
]

print("GroupedQueryAttention fields:", [n for n, _ in GroupedQueryAttention.__dict__.items() if not n.startswith("__")][:10], "...")
print("Recommended LoRA targets:", LORA_DEFAULT_TARGETS)


# ===== Dataset adaptation (GluonTS ListDataset) =====

class WindowDatasetLite(Dataset):
    """Convert (X, y) to GluonTS ListDataset for MoiraiPredictor (no shuffle)."""
    def __init__(self, X: np.ndarray):
        self.X = X.astype(np.float32)
    def __len__(self) -> int:
        return self.X.shape[0]
    def __getitem__(self, idx: int) -> Dict[str, Any]:
        target = self.X[idx].reshape(-1)
        return {"target": target.tolist(), "start": pd.Timestamp("2000-01-01")}


def make_list_dataset(X: np.ndarray) -> ListDataset:
    records = ({"target": x.reshape(-1).tolist(), "start": pd.Timestamp("2000-01-01")} for x in X)
    return ListDataset(records, freq="D")

# ===== LoRA config and injection utilities =====

def build_lora_config(r: int = 8, alpha: int = 16, dropout: float = 0.05, targets: Optional[List[str]] = None) -> LoraConfig:
    if targets is None:
        targets = LORA_DEFAULT_TARGETS
    return LoraConfig(
        r=r,
        lora_alpha=alpha,
        lora_dropout=dropout,
        bias="none",
        target_modules=targets,
        task_type=TaskType.FEATURE_EXTRACTION,
    )


def inject_lora_into_moirai(module: nn.Module, lora_cfg: LoraConfig) -> nn.Module:
    """Inject LoRA into MoiraiModule and return PEFT wrapper (do not change base forward signature)."""
    peft_wrapped = get_peft_model(module, lora_cfg)
    try:
        peft_wrapped.print_trainable_parameters()
    except Exception:
        pass
    return peft_wrapped


def adapt_peft_forward_to_moirai(peft_model: nn.Module) -> None:
    """Change PeftModel.forward to strictly accept Moirai-required fields and call base.forward directly.
    Do not bypass LoRA, as LoRA is already injected into the base module.
    """
    required = (
        "target", "observed_mask", "sample_id",
        "time_id", "variate_id", "prediction_mask", "patch_size",
    )
    hf_keys = {
        "input_ids", "inputs_embeds", "attention_mask", "labels",
        "decoder_input_ids", "decoder_attention_mask",
        "position_ids", "token_type_ids", "past_key_values",
        "use_cache", "output_attentions", "output_hidden_states", "return_dict",
    }

    def moirai_forward(self, *args, **kwargs):
        if len(args) == 1 and isinstance(args[0], dict):
            batch = dict(args[0])
        else:
            batch = dict(kwargs)

        for k in list(batch.keys()):
            if k in hf_keys:
                batch.pop(k, None)

        feed = {k: batch[k] for k in required if k in batch}
        base = getattr(self, "model", self)
        return base(**feed)

    peft_model.forward = moirai_forward.__get__(peft_model, peft_model.__class__)

class SafeForwardWrapper(nn.Module):
    def __init__(self, model: nn.Module):
        super().__init__()
        self.model = model

    def forward(self, *args, **kwargs):
        hf_keys = {
            "input_ids", "inputs_embeds", "attention_mask", "labels",
            "decoder_input_ids", "decoder_attention_mask",
            "position_ids", "token_type_ids", "past_key_values",
            "use_cache", "output_attentions", "output_hidden_states",
            "return_dict"
        }
        safe_kwargs = {k: v for k, v in kwargs.items() if k not in hf_keys}
        return self.model(*args, **safe_kwargs)

    def __getattr__(self, name: str):
        base_model = super().__getattribute__("model")
        return getattr(base_model, name)

HF_FILTER_KEYS = {
    "input_ids", "inputs_embeds", "attention_mask", "labels",
    "decoder_input_ids", "decoder_attention_mask",
    "position_ids", "token_type_ids", "past_key_values",
    "use_cache", "output_attentions", "output_hidden_states",
    "return_dict"
}

def _filter_hf_args_kwargs(args, kwargs):
    safe_kwargs = {k: v for k, v in kwargs.items() if k not in HF_FILTER_KEYS}
    if len(args) == 1 and isinstance(args[0], dict):
        d = dict(args[0])
        for k in list(HF_FILTER_KEYS):
            d.pop(k, None)
        return (d,), safe_kwargs
    return args, safe_kwargs

def wrap_forward_filter_inplace(module: nn.Module) -> None:
    """Wrap any nn.Module's forward in-place to filter HF-style keywords, without changing return value or gradients."""
    if not hasattr(module, 'forward'):
        return
    orig_forward = module.forward
    def filtered_forward(*args, **kwargs):
        a2, kw2 = _filter_hf_args_kwargs(args, kwargs)
        try:
            sig = inspect.signature(orig_forward)
            param_names = list(sig.parameters.keys())
            if len(a2) > 0 and len(kw2) > 0:
                for i, pname in enumerate(param_names[:len(a2)]):
                    if pname in kw2:
                        kw2.pop(pname, None)
        except Exception:
            pass
        return orig_forward(*a2, **kw2)
    module.forward = filtered_forward.__get__(module, module.__class__)


class PeftMoiraiAdapter(nn.Module):
    """Make PEFT model compatible with Uni2TS/Moirai forward signature:
    - Accepts inputs_embeds / input_ids and other HF arguments
    - Maps inputs_embeds to the expected argument name for MoiraiModule.forward (x/inputs/input/data/etc)
    - Removes unrelated HF keywords
    """
    def __init__(self, peft_model: nn.Module):
        super().__init__()
        self.peft_model = peft_model

    def forward(self, *args, **kwargs):
        inputs = kwargs.pop("inputs_embeds", None)
        hf_keys = {
            "input_ids", "attention_mask", "labels",
            "decoder_input_ids", "decoder_attention_mask",
            "position_ids", "token_type_ids", "past_key_values",
            "use_cache", "output_attentions", "output_hidden_states",
            "return_dict"
        }
        for k in list(kwargs.keys()):
            if k in hf_keys:
                kwargs.pop(k, None)

        base = self.peft_model

        if len(args) == 1 and isinstance(args[0], dict):
            batch = dict(args[0])
            if inputs is None and "inputs_embeds" in batch:
                inputs = batch.pop("inputs_embeds")
            for k in list(batch.keys()):
                if k in hf_keys:
                    batch.pop(k, None)
            args = (batch,)

        try:
            sig = inspect.signature(base.forward)
            param_names = list(sig.parameters.keys())
        except (ValueError, TypeError):
            param_names = []

        if inputs is not None:
            for candidate in ["x", "inputs", "input", "data", "ts", "values", "target", "targets"]:
                if candidate in param_names:
                    kwargs[candidate] = inputs
                    break
            else:
                try:
                    return base(inputs, *args, **kwargs)
                except TypeError:
                    kwargs["x"] = inputs

        return base(*args, **kwargs)

    def __getattr__(self, name: str):
        pm = super().__getattribute__("peft_model")
        return getattr(pm, name)


def patch_peft_forward_inplace(peft_model: nn.Module) -> None:
    """Wrap PEFT model's forward in-place:
    - Filter HF keywords (input_ids/inputs_embeds/etc)
    - If inputs_embeds exists, map it to the expected argument name for Moirai (x/inputs/input/data/...)
    """
    if not hasattr(peft_model, 'forward'):
        return
    orig_forward = peft_model.forward

    def safe_forward(*args, **kwargs):
        input_tensor = kwargs.pop('inputs_embeds', None)
        hf_keys = {
            'input_ids', 'attention_mask', 'labels',
            'decoder_input_ids', 'decoder_attention_mask',
            'position_ids', 'token_type_ids', 'past_key_values',
            'use_cache', 'output_attentions', 'output_hidden_states',
            'return_dict'
        }
        kwargs = {k: v for k, v in kwargs.items() if k not in hf_keys}

        if len(args) == 1 and isinstance(args[0], dict):
            batch = dict(args[0])
            if input_tensor is None and 'inputs_embeds' in batch:
                input_tensor = batch.pop('inputs_embeds')
            for k in list(batch.keys()):
                if k in hf_keys:
                    batch.pop(k, None)
            args = (batch,)

        if input_tensor is not None:
            try:
                base_mod = getattr(peft_model, 'model', peft_model)
                sig = inspect.signature(base_mod.forward)
                param_names = list(sig.parameters.keys())
            except (ValueError, TypeError):
                param_names = []
            mapped = False
            for candidate in ['x', 'inputs', 'input', 'data', 'ts', 'values']:
                if candidate in param_names:
                    kwargs[candidate] = input_tensor
                    mapped = True
                    break
            if not mapped:
                args = (input_tensor,) + args

        return orig_forward(*args, **kwargs)

    peft_model.forward = safe_forward.__get__(peft_model, peft_model.__class__)

def _count_lora_layers(model: nn.Module) -> int:
    count = 0
    for m in model.modules():
        if any(hasattr(m, attr) for attr in ["lora_A", "lora_B"]):
            count += 1
    return count

def report_lora_status(model: nn.Module, prefix: str = "[LoRA]") -> None:
    try:
        active = getattr(model, "active_adapters", None)
    except Exception:
        active = None
    num = _count_lora_layers(model)
    print(f"{prefix} active_adapters={active} | layers_with_lora={num}")

def _collect_lora_hits(model: nn.Module) -> Dict[str, int]:
    names = ["q_proj", "k_proj", "v_proj", "out_proj", "fc1", "fc2"]
    hits = {k: 0 for k in names}
    for name, module in model.named_modules():
        for k in names:
            if k in name and any(hasattr(module, attr) for attr in ["lora_A", "lora_B"]):
                hits[k] += 1
    return hits

def assert_lora_hits(model: nn.Module, require_attention: bool = True, require_ffn: bool = True, tag: str = "") -> None:
    """Assert whether LoRA hits attention and FFN target layers.
    - require_attention/require_ffn control whether hit is required
    """
    hits = _collect_lora_hits(model)
    attn_total = hits["q_proj"] + hits["k_proj"] + hits["v_proj"] + hits["out_proj"]
    ffn_total  = hits["fc1"] + hits["fc2"]
    print(f"[LoRA][hits]{'['+tag+']' if tag else ''} {hits} | attn={attn_total}, ffn={ffn_total}")
    if require_attention:
        assert attn_total > 0, "LoRA did not attach to any attention projections (q/k/v/out)"
    if require_ffn:
        assert ffn_total > 0, "LoRA did not attach to any FFN layers (fc1/fc2)"

def peek_first_batch(loader: DataLoader, model: nn.Module, pred_len: int = 1, window: Optional[int] = None):
    try:
        batch = next(iter(loader))
    except StopIteration:
        print("[batch] loader is empty")
        return
    try:
        print("[batch] keys:", list(batch.keys()) if isinstance(batch, dict) else type(batch))
        if isinstance(batch, dict) and "target" in batch:
            print("[batch] target.shape:", tuple(batch["target"].shape))
            if window is not None:
                print(f"[check] target last {pred_len} should be y; total length≈window({window})+{pred_len}")
    except Exception as e:
        print("[batch] inspect error:", repr(e))
    try:
        assert_lora_hits(model, True, True, tag="first_batch")
    except AssertionError as ae:
        print("[ASSERT][LoRA]", ae)

def load_datasets(npz_path: str) -> Dict[str, Any]:
    """Load .npz and return dict (keys include X_train_*, X_test_*, y_*, meta_*)."""
    return load_npz_dataset(npz_path)


def uni2ts_rolling_prediction(window_size: int, X_data: np.ndarray, batch_size: int = 256, prediction_length: int = 1,
                              num_samples: int = 100) -> np.ndarray:
    """Batch prediction using Moirai + LoRA (if available); used for daily signal backtest.
    - Prefer loading LoRA adapter for this window, otherwise fallback to base
    - Use median of samples as prediction
    """
    try:
        base = MoiraiModule.from_pretrained(
            BASE_MODEL_ID,
            torch_dtype=(torch.bfloat16 if USE_BF16 else None)
        )
    except Exception:
        base = MoiraiModule.from_pretrained(BASE_MODEL_ID)
        if USE_BF16:
            try:
                base = base.to(torch.bfloat16)
            except Exception:
                pass

    try:
        adapter_root = ADAPTER_ROOT
    except NameError:
        adapter_root = "/content/drive/MyDrive/uni2ts_large_portfolio(FineTuning)/uni2ts_results/adapters"
    adapter_dir = os.path.join(adapter_root, f"moirai_large_lora_w{window_size}", "lora_adapter")
    if os.path.isdir(adapter_dir):
        try:
            base = PeftModel.from_pretrained(base, adapter_dir)
            report_lora_status(base, prefix=f"[LoRA][infer][w{window_size}]")
            try:
                base = base.merge_and_unload()
                print(f"[LoRA] merged into base for inference (window={window_size})")
            except Exception:
                pass
            print(f"[LoRA] Loaded adapter for window={window_size}: {adapter_dir}")
        except Exception as e:
            print(f"[WARN] Failed to load LoRA adapter, use base. Error: {e}")

    try:
        base = base.to(torch.float32)
    except Exception:
        pass

    model = MoiraiForecast(
        module=base,
        prediction_length=prediction_length,
        context_length=window_size,
        patch_size=1,
        num_samples=num_samples,
        target_dim=1,
        feat_dynamic_real_dim=0,
        past_feat_dynamic_real_dim=0,
    )
    predictor = model.create_predictor(batch_size=batch_size, device=str(device))
    preds = batch_predict_with_moirai(predictor, X_data, batch_size=batch_size)
    return preds

# ===== Fine-tuning module (train LoRA per window and save) =====
DEBUG_FT = True
import lightning as L
from uni2ts.transform import (
    GetPatchSize, PatchCrop, PackFields, AddObservedMask, ImputeTimeSeries,
    DummyValueImputation, Patchify, AddVariateIndex, AddTimeIndex,
    MaskedPrediction, ExtendMask, FlatPackCollection, FlatPackFields,
    SequencifyField, SelectFields, FixedPatchSizeConstraints,
)
from uni2ts.data.loader import PadCollate

class MoiraiFinetuneDataset(Dataset):
    def __init__(self, X: np.ndarray, y: np.ndarray, transform, is_val: bool = False, eval_window: int = 0, pred_len: int = 1):
        self.X = X.astype(np.float32)
        self.y = y.astype(np.float32)
        self.transform = transform
        self.is_val = is_val
        self.eval_window = int(eval_window)
        self.pred_len = int(pred_len)
    def __len__(self) -> int:
        return len(self.X)
    def __getitem__(self, idx: int):
        import numpy as np
        x_1d = self.X[idx].reshape(-1)
        y_seq = np.atleast_1d(self.y[idx]).reshape(-1).astype(np.float32)
        if len(y_seq) < self.pred_len:
            pad = np.zeros(self.pred_len - len(y_seq), dtype=np.float32)
            y_seq = np.concatenate([y_seq, pad], axis=0)
        else:
            y_seq = y_seq[:self.pred_len]
        target_1d = np.concatenate([x_1d, y_seq], axis=0)
        entry = {"target": [target_1d], "freq": "D", "window": int(self.eval_window if self.is_val else 0)}
        out = self.transform(entry)
        if LAST_STEP_ONLY:
            pm = out.get("prediction_mask", None)
            if pm is not None:
                try:
                    pm[...] = 0
                except Exception:
                    import numpy as np
                    pm = np.zeros_like(pm)
                if pm.ndim == 1:
                    pm[-self.pred_len:] = 1
                elif pm.ndim == 2:
                    pm[-self.pred_len:, ...] = 1
                else:
                    pm[..., -self.pred_len:, :] = 1
                out["prediction_mask"] = pm

            om = out.get("observed_mask", None)
            if om is not None:
                if om.ndim == 1:
                    om[-self.pred_len:] = 0
                elif om.ndim == 2:
                    om[-self.pred_len:, ...] = 0
                else:
                    om[..., -self.pred_len:, :] = 0
                out["observed_mask"] = om
        return out

def finetune_one_window(window: int, output_root: str,
                        r: int = 8, alpha: int = 16, dropout: float = 0.05,
                        lr: float = 5e-4, weight_decay: float = 1e-2,
                        train_epochs: int = 2, warmup_ratio: float = 0.1,
                        batch_size: int = 1024) -> str:
    """Fine-tune one window and return the saved LoRA directory."""
    X_train, y_train, meta_train = extract_split(data_all, window, split="train")
    (X_tr, y_tr), (X_va, y_va), _ = time_based_val_split(X_train, y_train, meta_train, val_ratio=0.2)

    series_len = int(X_tr.shape[1])

    steps_per_epoch = max(1, len(X_tr) // batch_size)
    num_training_steps = steps_per_epoch * train_epochs
    num_warmup_steps = int(num_training_steps * warmup_ratio)

    try:
        module = MoiraiModule.from_pretrained(
            BASE_MODEL_ID,
            torch_dtype=(torch.bfloat16 if USE_BF16 else None)
        )
    except Exception:
        module = MoiraiModule.from_pretrained(BASE_MODEL_ID)
        if USE_BF16:
            try:
                module = module.to(torch.bfloat16)
            except Exception:
                pass
    lora_cfg = build_lora_config(r=r, alpha=alpha, dropout=dropout)
    module_lora = inject_lora_into_moirai(module, lora_cfg)
    adapt_peft_forward_to_moirai(module_lora)
    report_lora_status(module_lora, prefix="[LoRA][train]")
    assert_lora_hits(module_lora, require_attention=True, require_ffn=True, tag=f"train_w{window}")

    finetuner = MoiraiFinetune(
        min_patches=1,
        min_mask_ratio=0.2,
        max_mask_ratio=0.6,
        max_dim=max(64, series_len),
        num_training_steps=num_training_steps,
        num_warmup_steps=num_warmup_steps,
        module=module_lora,
        num_samples=32,
        lr=lr,
        weight_decay=weight_decay,
        log_on_step=False,
    )

    train_tf = finetuner.train_transform_map[None](
        distance=0,
        prediction_length=PRED_LEN,
        context_length=series_len,
        patch_size=1,
    )
    for t in getattr(train_tf, "transformations", []):
        if isinstance(t, GetPatchSize):
            if window in (252, 512):
                t.min_time_patches = 1
                t.patch_sizes = [1]
                t.patch_size_constraints = FixedPatchSizeConstraints(start=1, stop=1)
                print(f"[Train] window={window}, series_len={series_len}, fixed patch_size=[1]")
            else:
                model_max_patch = int(max(getattr(module_lora, "patch_sizes", [128])))
                upper = min(model_max_patch, series_len)
                valid = [p for p in range(1, upper + 1) if (series_len % p == 0)]
                if series_len <= 32:
                    valid = [1]
                if not valid:
                    valid = [1]
                t.min_time_patches = 1
                t.patch_sizes = valid
                t.patch_size_constraints = FixedPatchSizeConstraints(start=min(valid), stop=max(valid))
                print(f"[Train] window={window}, series_len={series_len}, valid_patch_sizes={valid}")
            break

    val_tf = finetuner.val_transform_map[None](
        offset=-PRED_LEN,
        distance=0,
        prediction_length=PRED_LEN,
        context_length=series_len,
        patch_size=1,
    )
    for t in getattr(val_tf, "transformations", []):
        if isinstance(t, GetPatchSize):
            if window in (252, 512):
                t.min_time_patches = 1
                t.patch_sizes = [1]
                t.patch_size_constraints = FixedPatchSizeConstraints(start=1, stop=1)
                print(f"[Val] window={window}, series_len={series_len}, fixed patch_size=[1]")
            else:
                t.min_time_patches = 1
                t.patch_sizes = [1]
                t.patch_size_constraints = FixedPatchSizeConstraints(start=1, stop=1)
                print(f"[Val] window={window}, series_len={series_len}, patch_size=[1]")
            break

    train_ds = MoiraiFinetuneDataset(X_tr, y_tr, train_tf, pred_len=PRED_LEN)
    val_ds   = MoiraiFinetuneDataset(X_va, y_va, val_tf, is_val=True, eval_window=0, pred_len=PRED_LEN)

    collate = PadCollate(max_length=series_len + PRED_LEN, seq_fields=tuple(finetuner.seq_fields))
    train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=False, collate_fn=collate)
    val_loader   = DataLoader(val_ds, batch_size=batch_size, shuffle=False, collate_fn=collate)

    if DEBUG_FT:
        try:
            print("[DEBUG] module_lora.forward signature:", inspect.signature(getattr(module_lora, "forward")))
        except Exception as e:
            print("[DEBUG] cannot inspect module_lora.forward:", repr(e))
        try:
            base_mod = getattr(module_lora, "model", None)
            if base_mod is not None and hasattr(base_mod, "forward"):
                print("[DEBUG] base model forward signature:", inspect.signature(base_mod.forward))
        except Exception as e:
            print("[DEBUG] cannot inspect base model forward:", repr(e))
        try:
            first_batch = next(iter(train_loader))
            if isinstance(first_batch, dict):
                print("[DEBUG] first train batch keys:", list(first_batch.keys()))
            else:
                print("[DEBUG] first train batch type:", type(first_batch))
            try:
                with torch.no_grad():
                    if isinstance(first_batch, dict):
                        module_lora(**first_batch)
                    else:
                        module_lora(first_batch)
                print("[DEBUG] dry-run forward passed")
            except Exception as fe:
                print("[DEBUG] dry-run forward error:", repr(fe))
                if isinstance(first_batch, dict):
                    for k, v in first_batch.items():
                        if torch.is_tensor(v):
                            print(f"[DEBUG]  - {k}: shape={tuple(v.shape)}, dtype={v.dtype}, device={v.device}")
                        else:
                            print(f"[DEBUG]  - {k}: type={type(v)}")
        except Exception as e:
            print("[DEBUG] failed to fetch first batch:", repr(e))
        try:
            peek_first_batch(train_loader, module_lora, pred_len=PRED_LEN, window=series_len)
        except Exception as e:
            print("[DEBUG] peek_first_batch error:", repr(e))

    ckpt_dir = os.path.join(output_root, f"moirai_large_lora_w{window}")
    os.makedirs(ckpt_dir, exist_ok=True)
    trainer = L.Trainer(
        max_epochs=train_epochs,
        default_root_dir=ckpt_dir,
        logger=False,
        enable_checkpointing=True,
        gradient_clip_val=1.0,
        accelerator=("gpu" if torch.cuda.is_available() else "cpu"),
        devices=1,
        precision=("bf16-mixed" if USE_BF16 else 32),
        num_sanity_val_steps=0,
    )
    trainer.fit(finetuner, train_loader, val_loader)

    lora_dir = os.path.join(ckpt_dir, "lora_adapter"); os.makedirs(lora_dir, exist_ok=True)
    try:
        if hasattr(finetuner.module, "save_pretrained"):
            finetuner.module.save_pretrained(lora_dir)
        elif hasattr(finetuner.module, "model") and hasattr(finetuner.module.model, "save_pretrained"):
            finetuner.module.model.save_pretrained(lora_dir)
        else:
            raise AttributeError("save_pretrained not found on module")
    except Exception:
        target = getattr(finetuner.module, "model", finetuner.module)
        torch.save({"state_dict": target.state_dict()}, os.path.join(lora_dir, "adapter_state.pt"))
    print(f"[Saved] LoRA adapter: {lora_dir}")
    return lora_dir

GroupedQueryAttention fields: ['_get_var_id', '_get_time_id', '_update_attn_mask', '_qk_proj', 'forward'] ...
Recommended LoRA targets: ['q_proj', 'k_proj', 'v_proj', 'out_proj', 'fc1', 'fc2']


In [ ]:
# Main process: fine-tuning and inference (separated execution)
RESULT_ROOT = "/content/drive/MyDrive/uni2ts_large_portfolio(FineTuning)/uni2ts_results"
ADAPTER_ROOT = os.path.join(RESULT_ROOT, "adapters")
PRED_ROOT = os.path.join(RESULT_ROOT, "predictions")
METRIC_CSV = os.path.join(RESULT_ROOT, "moirai_large_lora_metrics.csv")
for d in [RESULT_ROOT, ADAPTER_ROOT, PRED_ROOT]:
    os.makedirs(d, exist_ok=True)

DO_FINETUNE = True
if DO_FINETUNE:
    per_window_cfg = {
        5:   dict(r=8, alpha=16, dropout=0.05, lr=6e-4, weight_decay=1e-2, train_epochs=5, batch_size=256),
        21:  dict(r=8, alpha=16, dropout=0.05, lr=6e-4, weight_decay=1e-2, train_epochs=5, batch_size=256),
        252: dict(r=16, alpha=32, dropout=0.07, lr=4e-4, weight_decay=1e-2, train_epochs=5, batch_size=32),
        512: dict(r=16, alpha=32, dropout=0.08, lr=3e-4, weight_decay=1e-2, train_epochs=5, batch_size=15),
    }
    saved_adapters = {}

    for ws in WINDOWS:
        cfg = per_window_cfg[ws]
        print(f"\n===== Fine-tuning LoRA for window={ws} =====")
        adapter_dir = finetune_one_window(
            ws, ADAPTER_ROOT,
            r=cfg["r"], alpha=cfg["alpha"], dropout=cfg["dropout"],
            lr=cfg["lr"], weight_decay=cfg["weight_decay"],
            train_epochs=cfg["train_epochs"], batch_size=cfg["batch_size"],
        )
        saved_adapters[ws] = adapter_dir
    print("Saved adapters:", saved_adapters)


===== Fine-tuning LoRA for window=5 =====


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/684 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.24G [00:00<?, ?B/s]

trainable params: 3,016,704 || all params: 313,987,328 || trainable%: 0.9608
[LoRA][train] active_adapters=['default'] | layers_with_lora=144
[LoRA][hits][train_w5] {'q_proj': 24, 'k_proj': 24, 'v_proj': 24, 'out_proj': 24, 'fc1': 24, 'fc2': 24} | attn=96, ffn=48
[Train] window=5, series_len=5, valid_patch_sizes=[1]
[Val] window=5, series_len=5, patch_size=[1]
[DEBUG] module_lora.forward signature: (*args, **kwargs)
[DEBUG] base model forward signature: (target: jaxtyping.Float[Tensor, '*batch seq_len max_patch'], observed_mask: jaxtyping.Bool[Tensor, '*batch seq_len max_patch'], sample_id: jaxtyping.Int[Tensor, '*batch seq_len'], time_id: jaxtyping.Int[Tensor, '*batch seq_len'], variate_id: jaxtyping.Int[Tensor, '*batch seq_len'], prediction_mask: jaxtyping.Bool[Tensor, '*batch seq_len'], patch_size: jaxtyping.Int[Tensor, '*batch seq_len']) -> torch.distributions.distribution.Distribution
[DEBUG] first train batch keys: ['target', 'observed_mask', 'time_id', 'variate_id', 'prediction_

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

INFO: `Trainer.fit` stopped: `max_epochs=5` reached.
INFO:lightning.pytorch.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=5` reached.


[Saved] LoRA adapter: /content/drive/MyDrive/uni2ts_large_portfolio(FineTuning)/uni2ts_results/adapters/moirai_large_lora_w512/lora_adapter
Saved adapters: {5: '/content/drive/MyDrive/uni2ts_large_portfolio(FineTuning)/uni2ts_results/adapters/moirai_large_lora_w5/lora_adapter', 21: '/content/drive/MyDrive/uni2ts_large_portfolio(FineTuning)/uni2ts_results/adapters/moirai_large_lora_w21/lora_adapter', 252: '/content/drive/MyDrive/uni2ts_large_portfolio(FineTuning)/uni2ts_results/adapters/moirai_large_lora_w252/lora_adapter', 512: '/content/drive/MyDrive/uni2ts_large_portfolio(FineTuning)/uni2ts_results/adapters/moirai_large_lora_w512/lora_adapter'}


In [ ]:

# Module 2: Inference and Evaluation
# Inference alignment: median-of-samples, num_samples=100 by default
def batch_predict_with_moirai(predictor, X: np.ndarray, batch_size: int = 256) -> np.ndarray:
    preds = np.zeros(len(X), dtype=np.float32)
    for i in tqdm(range(0, len(X), batch_size), desc="Predicting"):
        end = min(i + batch_size, len(X))
        batch_records = (
            {"target": X[j].reshape(-1).tolist(), "start": "2000-01-01"}
            for j in range(i, end)
        )
        ds = ListDataset(batch_records, freq="D")
        forecasts = list(predictor.predict(ds))
        for j, f in enumerate(forecasts):
            if hasattr(f, "samples") and f.samples is not None:
                preds[i + j] = float(np.median(f.samples[:, 0]))
            else:
                try:
                    preds[i + j] = float(f.quantile(0.5)[0])
                except Exception:
                    preds[i + j] = float(getattr(f, "mean", [0.0])[0])
    return preds

def run_uni2ts_portfolio_backtest(start_year=2016, end_year=2024, window_sizes=None, model_names=None,
                                           npz_path="/content/drive/MyDrive/ERP Data/all_window_datasets_unscaled.npz"):
    """
    Portfolio simulation (daily prediction, next-day rebalance):
        1. Use Uni2TS Large model for zero-shot prediction
        2. Daily prediction to daily signal
        3. Daily portfolio construction (T+1 rebalance, strict permno alignment)
        4. Separate summary metrics and time series data
    """
    if window_sizes is None:
        window_sizes = [5, 21, 252, 512]
    if model_names is None:
        model_names = ["uni2ts large"]

    print(f"Starting Daily Rebalance Portfolio Backtesting Simulation ({start_year}-{end_year}) [Uni2TS Large]")

    backtester = PortfolioBacktester()
    datasets = load_datasets(npz_path)

    summary_results = []
    daily_series_data = []
    pred_rows = []

    WEIGHT_SCHEMES = ["VW", "EW"]

    for window in window_sizes:
        print(f"Processing window size: {window}")

        X_test = datasets[f"X_test_{window}"]
        y_test = datasets[f"y_test_{window}"]
        meta_raw = datasets.get(f"meta_test_{window}")
        if meta_raw is None:
            meta_test = pd.DataFrame({"PERMNO": np.arange(len(X_test))})
        else:
            meta_test = pd.DataFrame(meta_raw.item()) if hasattr(meta_raw, "item") else pd.DataFrame(meta_raw)

        permnos_test = meta_test["PERMNO"].values
        meta_test["signal_date"]  = pd.to_datetime(meta_test["date"])
        meta_test["ret_date"]     = pd.to_datetime(meta_test["ret_date"])
        market_caps = meta_test.get("MKTCAP", np.ones(len(permnos_test)))

        meta_test['date'] = pd.to_datetime(meta_test["date"])
        dates_test = meta_test['signal_date']

        for model_name in model_names:
            for scheme in WEIGHT_SCHEMES:
                all_y_true   = []
                all_y_pred   = []
                all_permnos  = []
                all_meta     = []
                print(f"  Model: {model_name}, Scheme: {scheme}")

                portfolio_daily_data = {
                    'long_only': {'returns': [], 'turnovers': [], 'dates': []},
                    'short_only': {'returns': [], 'turnovers': [], 'dates': []},
                    'long_short': {'returns': [], 'turnovers': [], 'dates': []}
                }

                prev_portfolio_data = {'long_only': None, 'short_only': None, 'long_short': None}

                signals_buf = {}

                for year in range(start_year, min(end_year + 1, 2025)):
                    print(f"  Processing year: {year}")

                    year_mask = (dates_test.dt.year == year)
                    if not np.any(year_mask):
                        continue

                    X_year = X_test[year_mask]
                    y_year = y_test[year_mask]
                    permnos_year = permnos_test[year_mask]
                    market_caps_year = market_caps[year_mask]
                    dates_year = dates_test[year_mask]
                    ret_dates_year = meta_test.loc[year_mask, 'ret_date'].values

                    batch_size = get_batch_size(window)
                    predictions_year = uni2ts_rolling_prediction(
                        window_size=window,
                        X_data=X_year,
                        batch_size=batch_size,
                        prediction_length=1
                    )

                    df_quarter = pd.DataFrame({
                        'signal_date': dates_year,
                        'ret_date': ret_dates_year,
                        'permno': permnos_year,
                        'market_cap': market_caps_year,
                        'actual_return': y_year,
                        'prediction': predictions_year
                    })

                    if scheme == 'VW':
                        df_q_save = df_quarter[['signal_date','ret_date','permno',
                                                'actual_return','prediction','market_cap']].copy()
                        df_q_save.rename(columns={'actual_return':'y_true',
                                                  'prediction':'y_pred'}, inplace=True)
                        df_q_save['model']  = model_name
                        df_q_save['window'] = window
                        pred_rows.append(df_q_save)

                    all_y_true.append(df_quarter['actual_return'].values)
                    all_y_pred.append(df_quarter['prediction'].values)
                    all_permnos.append(df_quarter['permno'].values)
                    all_meta.append(meta_test.loc[year_mask, :])

                    for signal_date, sig_grp in df_quarter.groupby('signal_date'):

                        daily_signals = (
                            sig_grp.groupby('permno')['prediction'].mean()
                                  .to_frame('prediction')
                                  .join(sig_grp.groupby('permno')['market_cap'].mean())
                        )
                        signals_buf[signal_date] = daily_signals

                        prev_date = signal_date - pd.tseries.offsets.BDay(1)
                        if prev_date not in signals_buf:
                            continue

                        sigs = signals_buf.pop(prev_date)

                        ret_grp = df_quarter[df_quarter['ret_date'] == signal_date]
                        if len(ret_grp) == 0:
                            continue

                        daily_actual_returns = (
                            ret_grp.groupby('permno')['actual_return']
                                   .mean()
                                   .reindex(sigs.index, fill_value=0)
                                   .values
                        )
                        daily_permnos = sigs.index.values

                        portfolios_data = backtester.create_portfolios_with_permno_tracking(
                            signals      = sigs['prediction'].values,
                            market_caps  = sigs['market_cap'].values,
                            permnos      = daily_permnos,
                            weight_scheme= scheme
                        )

                        for portfolio_type in ['long_only', 'short_only', 'long_short']:
                            portfolio_info = portfolios_data[portfolio_type]

                            portfolio_return, aligned_returns = backtester.calculate_aligned_portfolio_return(
                                portfolio_weights=portfolio_info['weights'],
                                portfolio_permnos=portfolio_info['permnos'],
                                actual_returns=daily_actual_returns,
                                actual_permnos=daily_permnos
                            )

                            if prev_portfolio_data[portfolio_type] is not None:
                                prev_w_ser = pd.Series(
                                    prev_portfolio_data[portfolio_type]['weights'],
                                    index=prev_portfolio_data[portfolio_type]['permnos']
                                )
                                cur_w_ser = pd.Series(
                                    portfolio_info['weights'],
                                    index=portfolio_info['permnos']
                                )

                                prev_r_ser = pd.Series(
                                    prev_portfolio_data[portfolio_type]['aligned_returns'],
                                    index=prev_portfolio_data[portfolio_type]['permnos']
                                )

                                aligned_prev_w = prev_w_ser.reindex(cur_w_ser.index, fill_value=0).values
                                aligned_prev_r = prev_r_ser.reindex(cur_w_ser.index, fill_value=0).values

                                aligned_cur_w = cur_w_ser.values

                                turnover = backtester.calc_turnover(
                                    w_t  = aligned_prev_w,
                                    r_t  = aligned_prev_r,
                                    w_tp1= aligned_cur_w
                                )
                            else:
                                turnover = np.sum(np.abs(portfolio_info['weights']))

                            portfolio_daily_data[portfolio_type]['returns'].append(portfolio_return)
                            portfolio_daily_data[portfolio_type]['turnovers'].append(turnover)
                            portfolio_daily_data[portfolio_type]['dates'].append(signal_date)

                            prev_portfolio_data[portfolio_type] = {
                                'weights'        : portfolio_info['weights'],
                                'permnos'        : portfolio_info['permnos'],
                                'aligned_returns': aligned_returns
                            }

                for portfolio_type in ['long_only', 'short_only', 'long_short']:
                    portfolio_data = portfolio_daily_data[portfolio_type]

                    if len(portfolio_data['returns']) > 0:
                        metrics = backtester.calculate_metrics(
                            returns=portfolio_data['returns'],
                            turnover_series=portfolio_data['turnovers']
                        )

                        rets = np.array(portfolio_data['returns'])
                        tovs = np.array(portfolio_data['turnovers'])

                        for tc in TC_GRID:
                            tag = TC_TAG[tc]
                            adj = rets - tovs * tc

                            ann_ret = adj.mean() * 252
                            ann_vol = adj.std(ddof=1) * np.sqrt(252)
                            sharpe  = ann_ret / ann_vol if ann_vol > 0 else 0

                            cum_adj = np.cumprod(1 + adj)
                            mdd = ((cum_adj - np.maximum.accumulate(cum_adj)) /
                                   np.maximum.accumulate(cum_adj)).min()

                            metrics[f'{tag}_annual_return'] = ann_ret
                            metrics[f'{tag}_annual_vol']    = ann_vol
                            metrics[f'{tag}_sharpe']        = sharpe
                            metrics[f'{tag}_max_drawdown']  = mdd

                        summary_results.append({
                            'scheme': scheme,
                            'model': model_name,
                            'window': window,
                            'portfolio_type': portfolio_type,
                            **metrics
                        })

                        rets_arr = np.array(portfolio_data['returns'])
                        tovs_arr = np.array(portfolio_data['turnovers'])
                        cum_no_tc = np.log1p(rets_arr).cumsum()

                        tc_ret_dict = {}
                        tc_cum_dict = {}
                        for tc in TC_GRID:
                            tag = TC_TAG[tc]
                            r = rets_arr - tovs_arr * tc
                            tc_ret_dict[tag] = r
                            tc_cum_dict[tag] = np.log1p(r).cumsum()

                        for i, date in enumerate(portfolio_data['dates']):
                            row = {
                                'scheme'        : scheme,
                                'model'         : model_name,
                                'window'        : window,
                                'portfolio_type': portfolio_type,
                                'date'          : str(date),
                                'return'        : rets_arr[i],
                                'turnover'      : tovs_arr[i],
                                'cumulative'    : cum_no_tc[i],
                            }
                            for tag in TC_TAG.values():
                                row[f'{tag}_return']     = tc_ret_dict[tag][i]
                                row[f'{tag}_cumulative'] = tc_cum_dict[tag][i]

                            daily_series_data.append(row)

                if scheme == "VW" and len(all_y_true) > 0:
                    y_all    = np.concatenate(all_y_true)
                    yhat_all = np.concatenate(all_y_pred)
                    perm_all = np.concatenate(all_permnos)
                    meta_all = pd.concat(all_meta, ignore_index=True)

                    k = X_test.shape[1]

                    m1_metrics = overall_interval_metrics_method1(
                        y_all, yhat_all, k,
                        permnos_all=perm_all,
                        meta_all=meta_all
                    )

                    full_pred_df = pd.concat(pred_rows, ignore_index=True)
                    full_pred_df['signal_date'] = pd.to_datetime(full_pred_df['signal_date'], errors='coerce')

                    cur = full_pred_df.loc[
                        (full_pred_df['window'] == window) &
                        (full_pred_df['model'] == model_name),
                        ['signal_date', 'y_true', 'y_pred']
                    ].dropna()

                    if len(cur) >= 30:
                        mean_ic, t_ic, pos_ic, _ = calc_ic_daily(cur, method='spearman')
                    else:
                        mean_ic, t_ic, pos_ic = np.nan, np.nan, np.nan

                    m1_metrics['RankIC_mean']  = mean_ic
                    m1_metrics['RankIC_t']     = t_ic
                    m1_metrics['RankIC_pos%']  = pos_ic

                    save_metrics(m1_metrics, name=model_name, window=window,
                        path="/content/drive/MyDrive/uni2ts_large_portfolio(FineTuning)/uni2ts_results/portfolio_metrics.csv")

    summary_df = pd.DataFrame(summary_results)
    daily_df = pd.DataFrame(daily_series_data) if daily_series_data else pd.DataFrame()

    tc_columns = [c for c in summary_df.columns if c.startswith('tc')]
    summary_df[tc_columns] = summary_df[tc_columns].fillna(0.0)

    def save_split_by_scheme(df, base_filename):
        """Helper function to save files split by scheme"""
        if df.empty:
            print(f"Warning: DataFrame is empty, skipping save for {base_filename}")
            return None, None

        vw_df = df[df['scheme'] == 'VW']
        ew_df = df[df['scheme'] == 'EW']

        vw_filename = f"/content/drive/MyDrive/uni2ts_large_portfolio(FineTuning)/uni2ts_results/{base_filename}_VW.csv"
        ew_filename = f"/content/drive/MyDrive/uni2ts_large_portfolio(FineTuning)/uni2ts_results/{base_filename}_EW.csv"

        vw_df.to_csv(vw_filename, index=False)
        ew_df.to_csv(ew_filename, index=False)

        print(f"VW results saved to {vw_filename}")
        print(f"EW results saved to {ew_filename}")

        return vw_filename, ew_filename

    save_split_by_scheme(summary_df, "portfolio_results_daily_rebalance")

    if not daily_df.empty:
        save_split_by_scheme(daily_df, "portfolio_daily_series")

    if pred_rows:
        pred_df = pd.concat(pred_rows, ignore_index=True)
        pred_df.to_csv("/content/drive/MyDrive/uni2ts_large_portfolio(FineTuning)/uni2ts_results/predictions_daily.csv", index=False)
        print(f"Saved {len(pred_df)} prediction rows to predictions_daily.csv")

    print(f"Generated {len(summary_results)} portfolio summary records")
    print(f"Generated {len(daily_series_data)} daily series records")

    return summary_df, daily_df, backtester

import os

results_dir = "/content/drive/MyDrive/uni2ts_large_portfolio(FineTuning)/uni2ts_results"
figures_dir = "/content/drive/MyDrive/uni2ts_large_portfolio(FineTuning)/uni2ts_figures"

os.makedirs(results_dir, exist_ok=True)
os.makedirs(figures_dir, exist_ok=True)

print("Starting Uni2TS Large Portfolio Backtesting...")

summary_results, daily_series, backtester = run_uni2ts_portfolio_backtest(
    start_year=START_YEAR,
    end_year=2024,
    window_sizes=WINDOW_SIZES
)

print("\n" + "="*60)
print("UNI2TS LARGE PORTFOLIO BACKTESTING RESULTS")
print("="*60)

print("\nSummary Results:")
print(summary_results.round(4))


Starting Uni2TS Large Portfolio Backtesting...
Starting Daily Rebalance Portfolio Backtesting Simulation (2016-2024) [Uni2TS Large]
Processing window size: 5
  Model: uni2ts large, Scheme: VW
  Processing year: 2016
[LoRA][infer][w5] active_adapters=['default'] | layers_with_lora=144
[LoRA] merged into base for inference (window=5)
[LoRA] Loaded adapter for window=5: /content/drive/MyDrive/uni2ts_large_portfolio(FineTuning)/uni2ts_results/adapters/moirai_large_lora_w5/lora_adapter


Predicting: 100%|██████████| 13/13 [00:05<00:00,  2.49it/s]


  Processing year: 2017
[LoRA][infer][w5] active_adapters=['default'] | layers_with_lora=144
[LoRA] merged into base for inference (window=5)
[LoRA] Loaded adapter for window=5: /content/drive/MyDrive/uni2ts_large_portfolio(FineTuning)/uni2ts_results/adapters/moirai_large_lora_w5/lora_adapter


Predicting: 100%|██████████| 13/13 [00:05<00:00,  2.51it/s]


  Processing year: 2018
[LoRA][infer][w5] active_adapters=['default'] | layers_with_lora=144
[LoRA] merged into base for inference (window=5)
[LoRA] Loaded adapter for window=5: /content/drive/MyDrive/uni2ts_large_portfolio(FineTuning)/uni2ts_results/adapters/moirai_large_lora_w5/lora_adapter


...

  Processing year: 2023
[LoRA][infer][w512] active_adapters=['default'] | layers_with_lora=144
[LoRA] merged into base for inference (window=512)
[LoRA] Loaded adapter for window=512: /content/drive/MyDrive/uni2ts_large_portfolio(FineTuning)/uni2ts_results/adapters/moirai_large_lora_w512/lora_adapter


Predicting: 100%|██████████| 194/194 [07:02<00:00,  2.18s/it]


  Processing year: 2024
[LoRA][infer][w512] active_adapters=['default'] | layers_with_lora=144
[LoRA] merged into base for inference (window=512)
[LoRA] Loaded adapter for window=512: /content/drive/MyDrive/uni2ts_large_portfolio(FineTuning)/uni2ts_results/adapters/moirai_large_lora_w512/lora_adapter


Predicting: 100%|██████████| 194/194 [07:03<00:00,  2.18s/it]


VW results saved to /content/drive/MyDrive/uni2ts_large_portfolio(FineTuning)/uni2ts_results/portfolio_results_daily_rebalance_VW.csv
EW results saved to /content/drive/MyDrive/uni2ts_large_portfolio(FineTuning)/uni2ts_results/portfolio_results_daily_rebalance_EW.csv
VW results saved to /content/drive/MyDrive/uni2ts_large_portfolio(FineTuning)/uni2ts_results/portfolio_daily_series_VW.csv
EW results saved to /content/drive/MyDrive/uni2ts_large_portfolio(FineTuning)/uni2ts_results/portfolio_daily_series_EW.csv
Saved 443400 prediction rows to predictions_daily.csv
Generated 24 portfolio summary records
Generated 52272 daily series records
\n============================================================
UNI2TS LARGE PORTFOLIO BACKTESTING RESULTS
\nSummary Results:
   scheme         model  window portfolio_type  annual_return  annual_vol  \
0      VW  uni2ts large       5      long_only         0.0855      0.2062   
1      VW  uni2ts large       5     short_only        -0.2116      0.2192   


In [ ]:

# ---------- Main function for 5-factor regression -----------
def run_factor_regression(port_ret, factors, use_excess=True):
    df = pd.concat([port_ret, factors], axis=1, join='inner').dropna()
    df.columns = ['ret'] + list(factors.columns)

    if use_excess:
        y = df['ret'].values
    else:
        y = df['ret'].values - df['rf'].values

    X = df[['mktrf','smb','hml','rmw','cma','umd']].values
    X = sm.add_constant(X)

    model = sm.OLS(y, X)
    res = model.fit()
    alpha = res.params[0]          # daily alpha
    resid_std = res.resid.std(ddof=1)

    ir_daily = alpha / resid_std          # daily IR
    ir_annual = ir_daily * np.sqrt(252)   # annualized IR

    y_hat = np.asarray(res.fittedvalues)

    out = {
        'N_obs'            : len(y),
        'alpha_daily'      : alpha,
        'alpha_annual'     : alpha*252,
        't_alpha'          : res.tvalues[0],
        'IR_daily'         : ir_daily,
        'IR_annual'        : ir_annual,
        'R2_zero'          : r2_zero(y, y_hat),
    }

    factor_names = ['MKT','SMB','HML','RMW','CMA','UMD']
    for i, fac in enumerate(factor_names, start=1):
        out[f'beta_{fac}'] = res.params[i]
        out[f't_{fac}']    = res.tvalues[i]

    return out

# ---------- 3. Batch run (EW/VW, three portfolio types) ----------
def batch_factor_analysis(
    daily_df: pd.DataFrame,
    factors_path: str,
    scheme: str,
    tc_levels=(0, 5, 10, 20, 40),
    portfolio_types=('long_only','short_only','long_short'),
    model_filter=None,
    window_filter=None,
    gross_only=False,            # If True, only calculate tc=0
            out_dir='/content/drive/MyDrive/uni2ts_large_portfolio(FineTuning)/uni2ts_results/factor_IR_results',
):
    """
    Generate a CSV containing IR results.
    gross_only=True  → only tc=0; False → all tc_levels.
    """
    import os
    os.makedirs(out_dir, exist_ok=True)

    fac = (pd.read_csv(factors_path, parse_dates=['date'])
             .set_index('date')
             .sort_index())

    sub = daily_df[daily_df['scheme'] == scheme].copy()
    if model_filter is not None:
        sub = sub[sub['model'].isin(model_filter)]
    if window_filter is not None:
        sub = sub[sub['window'].isin(window_filter)]

    tc_iter = (0,) if gross_only else tc_levels
    results = []

    for (model, win, ptype), g in sub.groupby(['model','window','portfolio_type']):
        g = g.sort_values('date').set_index(pd.to_datetime(g['date']))

        for tc in tc_iter:
            col = 'return' if tc == 0 else f'tc{tc}_return'
            if col not in g.columns:
                continue
            port_ret = g[col]
            stats = run_factor_regression(port_ret, fac, use_excess=True)
            stats.update({
                'scheme'        : scheme,
                'model'         : model,
                'window'        : win,
                'portfolio_type': ptype,
                'tc_bps'        : tc,
            })
            results.append(stats)

    df_out = pd.DataFrame(results)[[
        'scheme','model','window','portfolio_type','tc_bps','N_obs',
        'alpha_daily','alpha_annual','t_alpha',
        'IR_daily','IR_annual','R2_zero',
        'beta_MKT','t_MKT','beta_SMB','t_SMB',
        'beta_HML','t_HML','beta_RMW','t_RMW',
        'beta_CMA','t_CMA','beta_UMD','t_UMD'
    ]]

    tag = 'gross' if gross_only else 'net'
    fname = f'5_factor_analysis_{scheme}_{tag}.csv'
    df_out.to_csv(os.path.join(out_dir, fname), index=False)
    print(f'[Saved] {fname} ')
    return df_out



def run_all_factor_tests(vw_csv="/content/drive/MyDrive/uni2ts_large_portfolio(FineTuning)/uni2ts_results/portfolio_daily_series_VW.csv",
                         ew_csv="/content/drive/MyDrive/uni2ts_large_portfolio(FineTuning)/uni2ts_results/portfolio_daily_series_EW.csv",
                         factor_csv="/content/drive/MyDrive/ERP Data/5_Factors_Plus_Momentum.csv",
                         save_dir="/content/drive/MyDrive/uni2ts_large_portfolio(FineTuning)/uni2ts_results",
                         y_is_excess=True,
                         hac_lags=5,
                         save_txt=True):
    vw_df = pd.read_csv(vw_csv)
    ew_df = pd.read_csv(ew_csv)

    vw_gross = batch_factor_analysis(
        vw_df, factor_csv, scheme='VW', gross_only=True)
    vw_net   = batch_factor_analysis(
        vw_df, factor_csv, scheme='VW', gross_only=False)

    ew_gross = batch_factor_analysis(
        ew_df, factor_csv, scheme='EW', gross_only=True)
    ew_net   = batch_factor_analysis(
        ew_df, factor_csv, scheme='EW', gross_only=False)

    return vw_gross, vw_net, ew_gross, ew_net


vw_gross, vw_net, ew_gross, ew_net = run_all_factor_tests()

# === File paths ===
rf_file = "/content/drive/MyDrive/ERP Data/CRSP_2016_2024_top50_with_exret.csv"
vw_file = "/content/drive/MyDrive/uni2ts_large_portfolio(FineTuning)/uni2ts_results/portfolio_daily_series_VW.csv"
ew_file = "/content/drive/MyDrive/uni2ts_large_portfolio(FineTuning)/uni2ts_results/portfolio_daily_series_EW.csv"

# === Load rf (risk-free rate) data ===

rf_df = pd.read_csv(rf_file, usecols=["date", "rf"])
rf_df["date"] = pd.to_datetime(rf_df["date"])
rf_dict = dict(zip(rf_df["date"], rf_df["rf"]))


def adjust_returns_with_rf_grouped(file_path, output_path):
    df = pd.read_csv(file_path)
    # Robust date parsing: try standard parsing first, fallback to dayfirst if failed
    df["date"] = pd.to_datetime(df["date"], errors="coerce")
    _mask = df["date"].isna()
    if _mask.any():
        df.loc[_mask, "date"] = pd.to_datetime(df.loc[_mask, "date"], dayfirst=True, errors="coerce")

    # Find all return columns (including tc5_return, tc10_return, etc.)
    return_cols = [col for col in df.columns if "return" in col and "cumul" not in col]

    # Enforce portfolio_type order to avoid groupby sorting issues
    order = ["long_only", "short_only", "long_short"]
    df["portfolio_type"] = pd.Categorical(df["portfolio_type"], categories=order, ordered=True)

    df_list = []
    # Group by strategy/model/window, add rf and recalculate cumulative returns
    for _, group in df.groupby(["scheme", "model", "window", "portfolio_type"], sort=False):
        group = group.sort_values("date").copy()
        for col in return_cols:
            # Add rf to each day's return
            group[col] = group.apply(lambda row: row[col] + rf_dict.get(row["date"], 0), axis=1)

            # Find the corresponding cumulative column (keep naming consistent with original table)
            cum_col = col.replace("return", "cumulative")
            group[cum_col] = np.log1p(group[col]).cumsum()
        df_list.append(group)

    # Merge results and output in order
    df_new = pd.concat(df_list).sort_values(["scheme", "model", "window", "portfolio_type", "date"])
    df_new.to_csv(output_path, index=False)
    print(f"Finish: {output_path}")

# === Process VW and EW files separately ===
adjust_returns_with_rf_grouped(vw_file, "/content/drive/MyDrive/uni2ts_large_portfolio(FineTuning)/uni2ts_results/portfolio_daily_series_VW_with_rf.csv")
adjust_returns_with_rf_grouped(ew_file, "/content/drive/MyDrive/uni2ts_large_portfolio(FineTuning)/uni2ts_results/portfolio_daily_series_EW_with_rf.csv")

# ========== Download S&P500 (2016-2024) ==========
sp500 = yf.download("^GSPC", start="2016-01-01", end="2024-12-31")
price_col = "Adj Close" if "Adj Close" in sp500.columns else "Close"
sp500["daily_return"] = sp500[price_col].pct_change().fillna(0)
sp500["cum_return"] = np.cumsum(np.log1p(sp500["daily_return"]))
sp500 = sp500[["cum_return"]]
sp500.index = pd.to_datetime(sp500.index)

# ========== Configuration ==========
files = [
    ("VW", "/content/drive/MyDrive/uni2ts_large_portfolio(FineTuning)/uni2ts_results/portfolio_daily_series_VW_with_rf.csv"),
    ("EW", "/content/drive/MyDrive/uni2ts_large_portfolio(FineTuning)/uni2ts_results/portfolio_daily_series_EW_with_rf.csv")
]
tc_levels = [0, 5, 10, 20, 40]      # Transaction costs (bps)
windows = [5, 21, 252, 512]         # Window sizes
strategies = ["long_only", "short_only", "long_short"]

output_dir = "/content/drive/MyDrive/uni2ts_large_portfolio(FineTuning)/uni2ts_figures"
os.makedirs(output_dir, exist_ok=True)

# Gray economic event periods
crisis_periods = [
    (datetime(2018, 6, 1), datetime(2019, 1, 1), "US-China Trade War"),
    (datetime(2020, 2, 1), datetime(2020, 7, 1), "COVID-19"),
    (datetime(2022, 2, 1), datetime(2022, 6, 1), "Russia-Ukraine War"),
    (datetime(2023, 1, 1), datetime(2023, 4, 1), "US Bank Crisis"),
]

def plot_comparison_styled(df, scheme, tc, window):
    plt.figure(figsize=(15, 12))
    model_names = df["model"].unique()
    colors = plt.cm.tab10(np.linspace(0, 1, len(model_names)))

    offset_step = 0.02

    for i, strat in enumerate(strategies, 1):
        ax = plt.subplot(3, 1, i)

        plt.plot(sp500.index, sp500["cum_return"],
                 color="black", lw=2.5, label="S&P500 (Total Return)", zorder=10)

        for idx, model_name in enumerate(model_names):
            sub = df[(df["window"] == window) &
                     (df["portfolio_type"] == strat) &
                     (df["model"] == model_name)].sort_values("date")
            if sub.empty:
                continue

            if tc == 0:
                ret_col = "return"
            else:
                ret_col = f"tc{tc}_return"

            if ret_col not in sub.columns:
                continue

            log_cum = np.cumsum(np.log1p(sub[ret_col].values))

            y_shift = idx * offset_step
            plt.plot(sub["date"], log_cum + y_shift,
                     label=f"{model_name} ({strat.replace('_',' ').title()})",
                     lw=2, color=colors[idx], alpha=0.9)

        for start, end, label in crisis_periods:
            ax.axvspan(start, end, color='grey', alpha=0.3)
            ax.text(start + pd.Timedelta(days=10),
                    ax.get_ylim()[1]*0.92, label, fontsize=8, color='grey')
        ax.xaxis.set_major_locator(mdates.YearLocator())
        ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y'))
        ax.set_ylabel("Cumulative log return (start = 0)")
        ax.set_title(f"{scheme} | Window={window} | Strategy={strat} | TC={tc} bps")
        ax.grid(alpha=0.3)
        plt.xticks(rotation=30)
        plt.legend(bbox_to_anchor=(1.04, 1), loc='upper left', fontsize=8)

    plt.tight_layout()
    fname = f"{scheme}_window{window}_TC{tc}.png"
    plt.savefig(os.path.join(output_dir, fname), dpi=300, bbox_inches='tight')
    plt.close()


# ========== Main loop to generate all figures ==========
for scheme, file_path in files:
    df = pd.read_csv(file_path)
    df["date"] = pd.to_datetime(df["date"])
    for tc in tc_levels:
        for window in windows:
            plot_comparison_styled(df, scheme, tc, window)

print(f"All figures have been generated and saved to: {output_dir}/")


# === 3. Load R²_zero from portfolio_metrics.csv ===
metrics_df = pd.read_csv("/content/drive/MyDrive/uni2ts_large_portfolio(FineTuning)/uni2ts_results/portfolio_metrics.csv")[ ["Model", "Window", "R²_zero"] ]
metrics_df.rename(columns={"Model": "model", "Window": "window"}, inplace=True)

# === 4. Process VW/EW files ===
for fname in [
    "/content/drive/MyDrive/uni2ts_large_portfolio(FineTuning)/uni2ts_results/portfolio_results_daily_rebalance_VW.csv",
    "/content/drive/MyDrive/uni2ts_large_portfolio(FineTuning)/uni2ts_results/portfolio_results_daily_rebalance_EW.csv"
]:
    df = pd.read_csv(fname)

    df = df.merge(metrics_df, on=["model", "window"], how="left")

    rows = []
    for _, row in df.iterrows():
        r2 = float(row["R²_zero"]) if not pd.isna(row["R²_zero"]) else 0.0
        if row["portfolio_type"] == "long_only":
            d_sr, sr_star = delta_sharpe(r2, SR_MKT_EX)
            row["ΔSharpe"]  = d_sr
            row["Sharpe*"]  = sr_star
            row["baseline"] = f"SPX_excess ({SR_MKT_EX:.2f})"
        else:
            d_sr, sr_star = delta_sharpe(r2, 0)
            row["ΔSharpe"]  = d_sr
            row["Sharpe*"]  = sr_star
            row["baseline"] = "cash (0)"
        rows.append(row)

    pd.DataFrame(rows).to_csv(fname, index=False)
    print(f"[Update] ΔSharpe has been written to {fname}")

[Saved] 5_factor_analysis_VW_gross.csv 
[Saved] 5_factor_analysis_VW_net.csv 
[Saved] 5_factor_analysis_EW_gross.csv 
[Saved] 5_factor_analysis_EW_net.csv 


/tmp/ipython-input-678029247.py:168: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  for _, group in df.groupby(["scheme", "model", "window", "portfolio_type"], sort=False):


Finish: /content/drive/MyDrive/uni2ts_large_portfolio(FineTuning)/uni2ts_results/portfolio_daily_series_VW_with_rf.csv


/tmp/ipython-input-678029247.py:168: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  for _, group in df.groupby(["scheme", "model", "window", "portfolio_type"], sort=False):
/tmp/ipython-input-678029247.py:189: FutureWarning: YF.download() has changed argument auto_adjust default to True
  sp500 = yf.download("^GSPC", start="2016-01-01", end="2024-12-31")
[*********************100%***********************]  1 of 1 completed

Finish: /content/drive/MyDrive/uni2ts_large_portfolio(FineTuning)/uni2ts_results/portfolio_daily_series_EW_with_rf.csv


All figures have been generated and saved to: /content/drive/MyDrive/uni2ts_large_portfolio(FineTuning)/uni2ts_figures/
[Update] ΔSharpe has been written to /content/drive/MyDrive/uni2ts_large_portfolio(FineTuning)/uni2ts_results/portfolio_results_daily_rebalance_VW.csv
[Update] ΔSharpe has been written to /content/drive/MyDrive/uni2ts_large_portfolio(FineTuning)/uni2ts_results/portfolio_results_daily_rebalance_EW.csv
